# RASA Chatbot 在 Kubernetes 中的部署实战

在这个文章中，我们将 RASA Chatbot Server 部署到一个 Kubernetes 集群中。我们将使用 AWS 作为基础平台，
在 AWS China Region，您可以选择使用 [nwcdlabs/kops-cn](https://github.com/nwcdlabs/kops-cn) 自建 K8S, 
在 AWS Global Region, 您可以直接选择使用托管的 K8S平台 [EKS](https://aws.amazon.com/eks/).

## 在容器中运行 RASA
需要了解如何在 K8S 中运行 RASA, 我们要首先了解如何在 docker 下运行 RASA 项目。RASA 支持将放置在

```shell script
docker run --env AWS_ACCESS_KEY_ID=<aws-access-key-id> \
--env AWS_SECRET_ACCESS_KEY=<aws-secret-access-key> \
--env AWS_DEFAULT_REGION=<aws-region> --env BUCKET_NAME=<bucket-name> \
-p 5005:5005 \
<docker-image>:latest \
run -m <model-name> \
--enable-api \
--remote-storage aws
```

## K8S 部署

1. 先将 RASA 的基础环境制作成基础镜像，基础镜像只包含 Corpus, 并不包含 model, 请参考 [Dockerfile](Dockerfile)
1. 上传镜像到 Amazon ECR
1. 上传训练好的模型文件到某一个 Amazon S3 Bucket
1. 创建一个 AK/SK, 建议选择只能读取某个 S3 某个 prefix 的权限
1. [k8s_deployment/](k8s_deployment/) 目录下提供了 RASA service 的部署 YAML 文件

```
- image: 057005827724.dkr.ecr.cn-northwest-1.amazonaws.com.cn/rasa-jw
env:
  - name: AWS_ACCESS_KEY_ID
    value: <aws-access-key-id>
  - name: AWS_SECRET_ACCESS_KEY
    value: <aws-secret-access-key>
  - name: AWS_DEFAULT_REGION
    value: cn-northwest-1
  - name: BUCKET_NAME
    value: rasa-dev
args: ["run", "-m", "20190823-222532.tar.gz", "--enable-api", "--remote-storage", "aws"]
```
